In [ ]:
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.linear_model import SGDClassifier
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
from utils import evaluate_model

In [ ]:
df = pd.read_csv('data\credit_card_transactions.csv')

In [ ]:
df_dropped = df.copy()
df_dropped = df_dropped.drop(columns=['Unnamed: 0','trans_date_trans_time','cc_num','first', 'last', 'gender', 'street', 'lat', 'long', 'dob','trans_num','merch_lat', 'merch_long', 'unix_time','city', 'state'])

In [ ]:
df_dropped['merch_zipcode'] = df_dropped['merch_zipcode'].fillna(df_dropped['zip'])

In [ ]:
df_copy = df_dropped.copy()
df_copy = df_copy[df_copy['amt'] < 10000]

In [ ]:
dummies = pd.get_dummies(df_copy['category'], drop_first=True, dtype=int)
df_copy = pd.concat([df_copy, dummies], axis=1)
df_copy = df_copy.drop(columns=['category'])

In [ ]:
scaler = StandardScaler()
df_scaled = df_copy.copy()
df_scaled[['amt', 'zip', 'city_pop', 'merch_zipcode']] = scaler.fit_transform(df_scaled[['amt', 'zip', 'city_pop', 'merch_zipcode']])
df_scaled.head()

In [ ]:
merch_freq = df_scaled['merchant'].value_counts()
df_scaled['merchant_encoded'] = df_scaled['merchant'].apply(lambda x: merch_freq[x])
df_scaled = df_scaled.drop(columns=['merchant'])

In [ ]:
job_freq = df_scaled['job'].value_counts()
df_scaled['job_encoded'] = df_scaled['job'].apply(lambda x: job_freq[x])
df_scaled = df_scaled.drop(columns=['job'])

In [ ]:
df_scaled.head()

In [ ]:
X = df_scaled.drop(columns=['is_fraud'])
y = df_scaled['is_fraud']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
LogReg = LogisticRegression(solver='saga',max_iter=5000)
evaluate_model(LogReg, X_train, X_test, y_train, y_test)

In [ ]:
SGD = SGDClassifier(loss = 'log_loss', max_iter=5000, tol=1e-3, random_state=42)
evaluate_model(SGD, X_train, y_train, X_test, y_test)

In [ ]:
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
evaluate_model(LogReg, X_train_resampled, X_test, y_train_resampled, y_test)

In [ ]:
smoteenn = SMOTEENN(random_state=42)
X_train_resampled, y_train_resampled = smoteenn.fit_resample(X_train, y_train)
evaluate_model(LogReg, X_train_resampled, X_test, y_train_resampled, y_test)